In [17]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [18]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [19]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [20]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [21]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [22]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:05,  1.28it/s]


In [23]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,8.138714,4.138714,0.000000,10.138714,-3.861286,0.138714,0.0,8.138714,...,10.138714,-3.861286,0.138714,0.0,8.138714,4.138714,0.000000,10.138714,-3.861286,0.138714
1,2025-01-22,0.0,8.138714,4.138714,0.000000,10.138714,-3.861286,0.138714,0.0,8.138714,...,10.138714,-3.861286,0.138714,0.0,8.138714,4.138714,0.000000,10.138714,-3.861286,0.138714
2,2025-01-23,0.0,2.976959,4.138714,-13.566723,3.637189,-3.861286,0.138714,0.0,-4.125131,...,-5.308297,-3.861286,0.138714,0.0,10.079049,4.138714,-13.566723,12.582676,-3.861286,0.138714
3,2025-01-24,0.0,2.976959,4.138714,-13.566723,3.637189,-3.861286,0.138714,0.0,-4.125131,...,-5.308297,-3.861286,0.138714,0.0,10.079049,4.138714,-13.566723,12.582676,-3.861286,0.138714
4,2025-01-25,0.0,2.976959,4.138714,-13.566723,3.637189,-3.861286,0.138714,0.0,-4.125131,...,-5.308297,-3.861286,0.138714,0.0,10.079049,4.138714,-13.566723,12.582676,-3.861286,0.138714


In [24]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [25]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [26]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://echelonin.wpenginepowered.com...",2026-01-22,2026-01-26,1029 LV,1.151303e-01,41.00,57.00,-16.00,-18.177787
1,"<a href=""https://jlpartners.com/nationwide-reg...",2026-01-26,2026-01-26,1037 RV,9.420325e-02,40.99,50.91,-9.92,-16.800133
2,"<a href=""https://angusreid.org/one-year-of-don...",2026-01-16,2026-01-20,1838 A,7.653122e-02,37.00,56.00,-19.00,-17.017021
3,"<a href=""https://napolitannews.org/jobapproval...",2026-01-14,2026-01-21,3000 RV,6.245133e-02,47.00,51.00,-4.00,-13.898408
4,"<a href=""https://www.reuters.com/data/trumps-a...",2026-01-23,2026-01-25,1139 A,5.973258e-02,38.00,59.00,-21.00,-17.026807
...,...,...,...,...,...,...,...,...,...
544,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,4.288565e-20,52.00,43.00,9.00,5.604549
545,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,3.724858e-20,47.00,41.00,6.00,9.973193
546,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,3.314060e-20,49.00,36.00,13.00,8.272980
547,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,3.233802e-20,51.90,45.30,6.60,-0.017801


In [27]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [28]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-22,Trade/tariffs,2226 A,1.471904e-02,34.0,57.0,-23.0,-24.293193
1,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-16,2026-01-18,Trade/tariffs,2201 RV,1.014479e-02,45.0,46.0,-1.0,-13.468217
2,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-09,2026-01-12,Trade/tariffs,2201 RV,6.400932e-03,43.0,48.0,-5.0,-17.468217
3,"<a href=""https://s3.documentcloud.org/document...",2026-01-09,2026-01-12,Trade/tariffs,1209 A,5.879997e-03,37.0,62.0,-25.0,-19.553898
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-09,2026-01-12,Trade/tariffs,1602 A,5.019327e-03,37.0,56.0,-19.0,-20.293193
...,...,...,...,...,...,...,...,...,...,...
1077,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,3.015776e-14,47.0,37.0,10.0,8.706807
1078,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,1.745305e-14,48.0,37.0,11.0,9.706807
1079,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,1.011701e-14,45.0,41.0,4.0,2.706807
1080,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,5.928444e-15,48.0,35.0,13.0,11.706807


In [29]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [30]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://echelonin.wpenginepowered.com...",2026-01-22,2026-01-26,1029 LV,1.952063e-01,49.0,44.0,-5.0,-7.156737
1,"<a href=""https://www.nytimes.com/2026/01/22/us...",2026-01-12,2026-01-17,1625 RV,9.719443e-02,48.0,43.0,-5.0,-5.520599
2,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-23,2026-01-26,1520 RV,9.573730e-02,43.0,38.0,-5.0,-5.153880
3,"<a href=""https://emersoncollegepolling.com/jan...",2026-01-17,2026-01-19,1000 LV,9.080211e-02,48.1,41.7,-6.4,-7.826841
4,"<a href=""https://www.reuters.com/data/which-pa...",2026-01-23,2026-01-25,906 RV,9.032700e-02,41.0,37.0,-4.0,-6.008673
...,...,...,...,...,...,...,...,...,...
273,"<a href=""https://napolitannews.org/posts/gener...",2025-01-15,2025-01-16,1000 RV,6.000147e-20,44.0,51.0,7.0,3.433675
274,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,4.319836e-20,40.0,48.0,8.0,2.926140
275,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,2.149065e-20,45.0,48.0,3.0,1.196945
276,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,1.738780e-21,44.6,48.4,3.8,2.790740


In [31]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)